*The dataset used in this example is distributed at https://doi.org/10.11588/data/TJNQZG under the [CC-BY-SA-4.0](https://creativecommons.org/licenses/by-sa/4.0/) license.*

# QuickStart: a minimal workflow with the AFwizard


## About this QuickStart

With the QuickStart, a new user should get AFwizard up and running from start to finish to get a first impression of what it is all about. 

We use the [sample data set for the AFwizard](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG), which consists of all necessary input data and sample files of data produced and used during the workflow. It also contains a [detailed description and explanation of all steps](https://heidata.uni-heidelberg.de/file.xhtml?persistentId=doi:10.11588/data/TJNQZG/DYFJVQ&version=2.0). This QuickStart introduces the minimal steps that are necessary to perform adaptive filtering.

The sample data should be downloaded into one folder without further hierarchy, and the notebook should be invoked in this folder, which thus functions as the `working directory`. All commands shown below can be used as presented, except for the commands in step 3, where the paths to proprietary filtering software must be set in the respective system environment. 

Before testing the Quickstart, the [Prerequisites discussed in the Documentation](https://afwizard.readthedocs.io/en/latest/intro.html#prerequisites) and the [Installation of the AFwizard as explained in the Documentation](https://afwizard.readthedocs.io/en/latest/intro.html#installing-and-using) must be set up.

Since all data in this QuickStart are stored and new data during the workflow are created in the same directory, explanations in the following codeblocks do not use subdirectories or paths. However, using subdirectories for different kinds of data is useful in the course of intensive work with the AFwizard. Furthermore, a sub directory for the final output will be created automatically if no option is given (see below "Step 5: Adaptive Filtering").

Before starting the workflow with the AFwizard, we need to import the `afwizard` library. Using the alias `af` in the code below allows us to use a short form in the following steps instead of `afwizard`.

In [ ]:
import afwizard as af

## Step 1: Prepare original point cloud and segment file

_Detailed explanations can be found in the documentation on [Working with LIDAR datasets in AFwizard](https://afwizard.readthedocs.io/en/latest/datasets.html)._

Load original point cloud (sample data: `nkd_pcl_epsg6670.laz`) of the whole region: 

In [ ]:
ds = af.DataSet(filename="nkd_pcl_epsg6670.laz", spatial_reference="EPSG:6670")

Remove existing classification from the point cloud (optional step), this may take a while:

In [ ]:
ds = af.remove_classification(ds)

Load segmentation layer and its spatial reference (sample data: `nkd_sgm.geojson`, EPSG:6670) where all segments cover the whole region and are classified according to environmental characteristics as "paddy", "ridge", "slope", or "valley".

In [ ]:
segmentation_overlay = af.load_segmentation(
    "nkd_sgm.geojson", spatial_reference="EPSG:6670"
)

None of the above commands should give any feedback to the user.

## Step 2: Create spatial samples (point clouds) for segment classes

_Detailed explanations can be found in the documentation on [Working with LIDAR datasets in AFwizard](https://afwizard.readthedocs.io/en/latest/datasets.html)._

We will now prepare samples from the point cloud such that each spatial sample represents the typical features of one segment class, either "paddy", "ridge", "slope", or "valley" in the case of the [sample data set](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG). 

The step of creating a spatial sample is called `restrict`, it is performed in a Graphical User Interface that is called as follows (it may take a while until the GUI shows up): 

In [ ]:
rds = ds.restrict(segmentation_overlay=segmentation_overlay)

If the optional step of removing the classification of point clouds has not been performed, the lower left part will inform about how many points of which class the data set contains. 

Clicking on the button "Visualize" on the left gives the option to load a standard visualization of the area, which can be modified with some options in the drop-down menu. The zoom level may be changed on the top right of the map. Clicking on the polygon in the left half of the map enables the user to create a subselection of a segment as representative of a segment class. 

In creating a spatial sample for a particular class, a polygon is drawn in a segment of that class. After clicking into the segment and finishing the polygon, we click on "Finalize" and close the GUI.

This spatial sample now has to be saved; to not overwrite the existing data, one may add a suffix "`_new` to the file name. Considering a spatial sample for segment class "valley" is created, not to overwrite the existing spatial sample `sample_pcl_valley.las`, the new selection may be called `sample_pcl_valley_new.las` like in the following dialog.

In [ ]:
saved = rds.save("sample_pcl_valley_new.las", overwrite=False)

We have now created and saved one point cloud data set that is representative of valleys in our original point cloud. The steps under "Step 2" have to be repeated for all other segment classes, such that we save four new files `..._new.las`.

The [sample data set](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG) already provides one spatial sample for each class (`sample_pcl_paddy.las`, `sample_pcl_ridge.las`, `sample_pcl_slope.las`, `sample_pcl_valley.las`), such that the steps below can be tested with these data sets instead of newly created data sets.

## Step 3: Creating filter pipelines

_Detailed explanations can be found in the documentation on [Working with filter libraries](https://afwizard.readthedocs.io/en/latest/libraries.html)._

We can now use these spatial samples to create filter pipelines for segment classes from scratch or to modify existing filter pipelines like those shipped with the [sample data set](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG) (`nkd_fpl_paddy_LT.json`, `nkd_fpl_slope_LT.json`, `nkd_fpl_valley_TF.json`). These were created based on the four `.las` files created as described in step 2 and are also provided with the [sample data set](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG). Since the segments of class "ridge" use identical settings to the segments of class "slope," the same filter pipeline can be used for slopes and ridges. Therefore, only the three filter pipelines mentioned above are shipped for four segment classes. 

However, these filter pipelines need the installation of OPALS and LAStools. Therefore, you can only use the filter pipelines if you have access to these tools. In that case, the path to each tool has to be set with the respective AFwizard commands.

**To perform these commands shown in the following two lines, the `#` sign at the beginning of each line must be deleted, and the correct path to the installations in the respective environment has to be set.**

In [ ]:
# af.set_opals_directory([path_to_opals])
# af.set_lastools_directory([path_to_lastools])

In this workflow, we will show the creation and modification of a filter pipeline for segments of the class "slope"; **the steps must be repeated for all segment classes**.

Whether we create a new filter pipeline from scratch or modify an existing one, we test the parameter settings with the respective spatial sample. In this workflow, we will use the point cloud sample for "slope" shipped with the sample data: `sample_pcl_slope.las` and declare the spatial reference, which is EPSG:6670 in our case.

In [ ]:
dataset = af.DataSet(filename="sample_pcl_slope.las", spatial_reference="EPSG:6670")

### Creating a filter pipeline from scratch

We will now create a filter pipeline for segments of class "slope" from scratch and load the GUI for pipeline tuning. A detailed description of the process in this specific example is given in the [documentation of the sample data](https://heidata.uni-heidelberg.de/file.xhtml?persistentId=doi:10.11588/data/TJNQZG/DYFJVQ&version=2.0).

The GUI for pipeline creation and pipeline tuning is evoked with the following command:

In [ ]:
pipeline = af.pipeline_tuning(dataset)

For testing the workflow, the input of useful filter parameters is less important than getting used to the options, but the paragraphs below briefly demonstrate some sensible options that finally led to the creation of the filter pipelines shipped with the [sample data set](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG). Tab numbers may differ. 

1. The data set is shown in tab \#0 just as loaded.
2. Raster resolution on the right is set to 0.2, and the hillshade parameters are modified such that the Angle Altitude is 45 degrees (45.30). Clicking the preview button shows tab \#1.
3. Another pipeline stage is selected on the left side; here we choose `lasground_new (LASTools)` with their default values and click the preview button; the result is shown in tab \#2.
4. To add more options to choose a better example, we create a range of parameters for the LASTools in order to later compare the results and select the best option.
    1. The parameter spike will be calculated with 0.4 increments between 0.1 and 0.9.
    2. For each of the thre values, thre parameter steps at 0.6, 1.2, 1.8 will be calculated.
    3. Clicking on the preview button calculated 9 candidates which are shown in tabs \#3 to \#11.
    4. The model in tab \#6 (spike 0.1; step 1.2 -- tab numbers may differ!) seems the best candidate. 
5. The new candidate will be further refined by adding a range to the parameter step from value 0.9 to 1.5 with 0.1 increments; this adds 6 more models which are shown in the tabs \#12 to \#17 after clicking the preview button.

Tabs that are of no use can be deleted with the button "Delete this filtering" on the right side; models that shall be preserved can be saved as point clouds, geotiffs or images using the buttons below the view in the center column. 

A log of all steps is shown below the GUI.

Finally, a setting for the filter pipeline has to be chosen, we decide for tab\#13 (offset 0; spike 0.1; step 1). Chosing the respective tab with these parameters, we can enter meta data in the lower left column and click the button `Finalize`. Entering meta data is imperative, otherwise the filter pipelinw will not be saved and a warning message will be displayed `WARNING: This filter has insufficient metadata. Please consider adding in af.pipeline_tuning!`.
For testing the workflow, the input of functional filter parameters is less critical than getting used to the options. Still, the paragraphs below briefly demonstrate some sensible options that finally led to creating the filter pipelines shipped with the [sample data set](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG). Tab numbers may differ.

1. The data set is shown in tab #0, just as loaded.
2. Raster resolution on the right is set to 0.2, and the hillshade parameters are modified such that the Angle Altitude is 45 degrees (45.30). Clicking the preview button shows tab #1.
3. Another pipeline stage is selected on the left side; here, we choose `lasground_new (LASTools)` with their default values and click the preview button; the result is shown in tab #2.
4. To add more options to choose a better example, we create a range of parameters for the LASTools to compare the results later and select the best option.
    1. The parameter spike will be calculated with 0.4 increments between 0.1 and 0.9.
    2. Three parameter steps at 0.6, 1.2, and 1.8 will be calculated for each of the three values.
    3. Clicking on the preview button calculated nine candidates shown in tabs #3 to #11.
    4. The model in tab #6 (spike 0.1; step 1.2 -- tab numbers may differ!) seems the best candidate.
5. The new candidate will be further refined by adding a range to the parameter step from value 0.9 to 1.5 with 0.1 increments; this adds six more models shown in tabs #12 to #17 after clicking the preview button.

Tabs that are of no use can be deleted with the button "Delete this filtering" on the right side; models that shall be preserved can be saved as point clouds, GeoTIFF, or images using the buttons below the view in the center column.

A log of all steps is shown below the GUI.

Finally, a setting for the filter pipeline has to be chosen; we decide on tab#13 (offset 0; spike 0.1; step 1). Selecting the respective tab with these parameters, we can enter metadata in the lower left column and click the button `Finalize`. Entering metadata is imperative. Otherwise, the filter pipeline will not be saved, and a warning message will be displayed: `WARNING: This filter has insufficient metadata. Please consider adding in af.pipeline_tuning!`.

The filter pipeline name may differ from the file name of the pipeline. -- In our [sample data set](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG), we have chosen to give the same name to the filter pipeline file name as to the filter pipeline itself (also called `pipeline title`), which is unnecessary.
The filter pipeline name may differ from the file name of the pipeline. -- In our [sample data set](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG) we have chosen to give the same name to the filter pipeline file name as to the filter pipeline itself (also called `pipeline title`) which is not necessary.

In [ ]:
af.save_filter(pipeline, "nkd_fpl_slope_LT_new.json")

### Modify existing filter pipelines

The dataset is loaded as shown at the beginning of step 3 above. Now we load an existing filter pipeline as `old_pipeline`; here, we will load the filter pipeline for segments of class "slope," which is `nkd_fpl_slope_LT.json` in the [sample data set](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG).

In [ ]:
old_pipeline = af.load_filter("nkd_fpl_slope_LT.json")

This pipeline will be edited in a GUI that is evoked with the following command. The steps resemble those in the creation of a filter pipeline from scratch.

In [ ]:
edited_pipeline = af.pipeline_tuning(dataset, pipeline=old_pipeline)

The new version is saved with the following command; we will add "modified" to the original file name in this case.

In [ ]:
af.save_filter(edited_pipeline, "nkd_fpl_slope_LT_modified.json")

## Step 4: Mapping segmentations to filter pipelines

_Detailed explanations can be found in the documentation on [Mapping segmentations to filter pipelines](https://afwizard.readthedocs.io/en/latest/segmentation.html)._

We load the file containing polygons representing the extent of segments and their segment class (`nkd_sgm.geojson` in the [sample data set](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG)). 

In [ ]:
segmentation = af.load_segmentation("nkd_sgm.geojson", spatial_reference="EPSG:6670")

The new version is saved with the following command; we will add "modified" to the original file name in this case:

With the following command, we make accessible the filter pipelines stored in the working directory. The [sample data set](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG) ships with only three filter pipelines since two segment classes ("ridge" and "slope") work well with the same filter pipeline (`nkd_fpl_slope_LT.json`). In the left column under "libraries," only the working directory may be selected, so the center columns show the three filter pipelines from the [sample data set](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG), which are stored in the working directory. 

Clicking on one filter pipeline shows the metadata in the right column. 

To select all three pipelines, we hold the `ctrl` key (Windows) or `cmd` key (Mac) and select all three filters. The Finalize button on top of the right column loads these filters for application in the next step.

In [ ]:
pipelines = af.select_pipelines_from_library()

Segment classes and filter pipelines can now be matched in a GUI, which is evoked with the following command.

On the top right, a drop-down menu shows options from all data fields in the GeoJSON file; in the case of the [sample data set](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG), there are unique ID fields (`id`) for each segment and class attribution for each segment in a `class` field. In the drop-down menu, select "class."

It should be noted that the field to select the class in the `geojson` file may differ from the name "class" chosen in the [sample data set](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG); likewise, the name of the filter pipeline may differ in the `geojson` file from the file name itself. In the [sample data set](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG), both names are identical. 

In [ ]:
assigned_segmentation = af.assign_pipeline(
    ds, segmentation=segmentation, pipelines=pipelines
)

Clicking the button "Visualize" will load a standard visualization, which is optional. 

Each segment will get a filter pipeline name (the name stored in the filter pipeline file, not the filename) assigned by choosing from the dropdown menu.

When all filter pipelines are assigned to a segment class, clicking the "Finalize" button closes the GUI.

The information will be saved in another GeoJSON file containing the same data as the original GeoJSON file and two more data fields: A hash value representing the filter pipeline settings chosen and an informational field that holds the actual name of the filter pipeline. The assigned segmentation file is saved with the command below.

In [ ]:
assigned_segmentation.save("nkd_sgm_assigned_TF_new.geojson")

The [sample data set](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG) ships with a file `nkd_sgm_assigned_TF.geojson` that can be used in the following step instead of the newly created file `nkd_sgm_assigned_TF_new.geojson`. 

## Step 5: Adaptive filtering

Finally, the adaptive filtering has to be calculated to create a new DTM. This is done on a command line/terminal basis and is the heart of the AFwizard. 

Many parameters may be set in this stage, and more detailed information can be found on the AFwizard documentation page [afwizard](https://afwizard.readthedocs.io/en/latest/execution.html#afwizard). Here we choose the bare minimum of options, and this step can be executed with the data provided in the [sample data set](https://heidata.uni-heidelberg.de/dataset.xhtml?persistentId=doi:10.11588/data/TJNQZG).

`afwizard --dataset=nkd_pcl_epsg6670.laz --dataset-crs=EPSG:6670 --segmentation=nkd_sgm_assigned_TF.geojson --segmentation-crs=EPSG:6670`

If the terminal is set to the working directory, the command above can be copied and pasted to invoke the AFwizard and the primary adaptive filtering process. 

During this step, an output directory is created as a subdirectory of the current working directory -- if no parameter is given for the output data. This directory holds the following files: 

1. The filter pipeline JSON files for complete documentation. In our case, these are three files. 
2. An output log-file `output.log`.
3. The DTM resulting from adaptive filtering:
    1. The point cloud data, in our case, `nkd_pcl_epgs6670_filtered.las`
    2. A multiband GeoTIFF file, in our case, `nkd_pcl_epgs6670_filtered.tiff`, where band 1 contains the DTM.

This step wraps up the minimal workflow with the Adaptive Filtering Wizard.